# MMVC学習用の音声データの録音

ver.2022/06/19



## 1 概要
MMVC学習用の自身の音声データを録音します。

## 2 Google Drive をマウント
**Google Drive にアップロードした MMVC_Trainer を参照できるように、設定します。**

「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」

といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

成功すれば、下記メッセージが出ます。
```
Mounted at /content/drive/
```


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

cdコマンドを実行して、マウントしたGoogle Drive のMMVC_Trainerディレクトリに移動します。

%cd 「MMVC_Trainerをgoogle driveにパップロードしたパス」

としてください。

正しいパスが指定されていれば

-rw------- 1 root root 11780 Mar  4 16:53 attentions.py

-rw------- 1 root root  4778 Mar  4 16:53 commons.py

drwx------ 2 root root  4096 Mar  5 15:20 configs

...といった感じに表示されるはずです。

In [ ]:
%cd /content/drive/MyDrive/
!ls -la

## 3 学習用音声を録音する

ITAコーパスを利用して音声を録音します。  
次の録音用のプログラムを読み込みます。

In [ ]:
# from https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
# from https://colab.research.google.com/github/espnet/notebook/blob/master/espnet2_asr_realtime_demo.ipynb

from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec, filename='audio.wav'):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec * 1000))
  b = b64decode(s.split(',')[1])
  with open(filename, 'wb+') as f:
    f.write(b)

import librosa
import librosa.display
import matplotlib.pyplot as plt
import pysndfile
from IPython.display import display, Audio
import warnings
warnings.filterwarnings('ignore')

def rec(sec, filename, text, hira):
  myvoice_dir = "./dataset/textful/00_myvoice/wav/"
  sampling_rate = 24000
  if sec > 15 :
    print("15秒以上の音声は学習できません。\n録音秒数を15秒以下に指定してください。")
    return
  print(f"{text}\n{hira}")
  record(sec, "temp.wav")
  print("---終了---")
  speech, rate = librosa.load("temp.wav", sr=sampling_rate)
  speecht, index = librosa.effects.trim(speech, top_db=30)
  pysndfile.sndio.write(myvoice_dir + filename, speecht, rate=rate, format='wav', enc='pcm16')
  librosa.display.waveplot(speecht, sr=rate)
  plt.show()
  display(Audio(speecht, rate=rate))


以下の100件の学習データを録音します。  
実行したら話す内容が表示されるので「---終了---」と表示されるまでに録音してください。  
録音した波形が表示されるので、再生してみて良ければ次の録音をします。  
うまく録音できていなかったら再度実行して録音しなおしてください。  
時々録音状態が終了しない時があり、その際は左の■で止めて、ページ自体をスーパーリロード（shiftキーを押しながら再読み込み）してから再度実行してください。  
録音時間が短い場合、先頭「rec(5, …」などの「5」の部分が録音秒数になっているので、修正してください。ただしMMVCの最大学習時間が15秒となっているのでそれ以下に指定してください。  
学習データは MMVC_Trainer/dataset/textful/00_myvoice/wav/ 以下に保存されます。

In [ ]:
rec(3, "emotion001.wav", "えっ嘘でしょ。", "えっうそでしょ。")

In [ ]:
rec(5, "emotion002.wav", "シュヴァイツァーは見習うべき人間です。", "しゅゔぁいつぁーわみならうべきにんげんです。")

In [ ]:
rec(5, "emotion003.wav", "デーヴィスさんはとても疲れているように見える。", "でーゔぃすさんわとてもつかれているよーにみえる。")

In [ ]:
rec(5, "emotion004.wav", "スティーヴはジェーンから手紙をもらった。", "すてぃーゔわじぇーんからてがみをもらった。")

In [ ]:
rec(10, "emotion005.wav", "彼女はモーツァルトやベートーヴェンといった、古典派の作曲家が好きだ。", "かのじょわもーつぁるとやべーとーゔぇんといった、こてんはのさっきょくかがすきだ。")

In [ ]:
rec(15, "emotion006.wav", "ストラットフォード・オン・エイヴォンは、シェイクスピアの生まれたところですが、毎年多くの観光客が訪れます。", "すとらっとふぉーどおんえいゔぉんわ、しぇいくすぴあのうまれたところですが、まいとしおおくのかんこーきゃくがおとずれます。")

In [ ]:
rec(5, "emotion007.wav", "彼はピューマを慣らすのに、大いに骨を折った。", "かれわぴゅーまをならすのに、おおいにほねをおった。")

In [ ]:
rec(5, "emotion008.wav", "彼が解雇されるとは妙な話だ。", "かれがかいこされるとわみょーなはなしだ。")

In [ ]:
rec(5, "emotion009.wav", "クリスはヴァンパイア・ナイトを倒した。", "くりすわゔぁんぱいあないとをたおした。")

In [ ]:
rec(5, "emotion010.wav", "彼のあだ名は言い得て妙だよね。", "かれのあだなわいいえてみょーだよね。")

In [ ]:
rec(15, "emotion011.wav", "イタリア旅行で彼は、いくつか景勝の地として有名な都市、例えば、ナポリやフィレンツェを訪れた。", "いたりありょこーでかれわ、いくつかけーしょーのちとしてゆうめーなとし、たとえば、なぽりやふぃれんつぇをおとずれた。")

In [ ]:
rec(10, "emotion012.wav", "ゼロという概念は、ヒンドゥー文化に由来している。", "ぜろとゆうがいねんわ、ひんどぅーぶんかにゆらいしている。")

In [ ]:
rec(5, "emotion013.wav", "そのいたずら娘は何食わぬ顔をした。", "そのいたずらむすめわなにくわぬかおをした。")

In [ ]:
rec(10, "emotion014.wav", "スミスさん、ピエール・デュボワをご紹介しますわ。私の親友なの。", "すみすさん、ぴえーるでゅぼわをごしょーかいしますわ。わたしのしんゆうなの。")

In [ ]:
rec(10, "emotion015.wav", "どーすんの、このお店。完ッ全に閑古鳥が鳴いちゃってるじゃない。", "どーすんの、このおみせ。かんっぜんにかんこどりがないちゃってるじゃない。")

In [ ]:
rec(5, "emotion016.wav", "頂上に着くと皆でヤッホーと叫んだ。", "ちょーじょーにつくとみんなでやっほーとさけんだ。")

In [ ]:
rec(5, "emotion017.wav", "あっベルが鳴ってる。", "あっべるがなってる。")

In [ ]:
rec(5, "emotion018.wav", "彼女は彼にぴょこんとおじぎをした。", "かのじょわかれにぴょこんとおじぎをした。")

In [ ]:
rec(5, "emotion019.wav", "男子生徒のめいめいが、帽子に学校のバッジをつけています。", "だんしせーとのめーめーが、ぼーしにがっこーのばっじをつけています。")

In [ ]:
rec(3, "emotion020.wav", "ヒューズが飛んだ。", "ひゅーずがとんだ。")

In [ ]:
rec(5, "emotion021.wav", "私はポピュラー音楽を聞きたい。", "わたしわぽぴゅらーおんがくをききたい。")

In [ ]:
rec(5, "emotion022.wav", "猫はにゃーにゃーと鳴く。", "ねこわにゃーにゃーとなく。")

In [ ]:
rec(10, "emotion023.wav", "私の一番上の兄が父の名代としてその会合に出席した。", "わたしのいちばんうえのあにがちちのみょーだいとしてそのかいごーにしゅっせきした。")

In [ ]:
rec(5, "emotion024.wav", "彼は首相にインタビューした。", "かれわしゅしょーにいんたびゅーした。")

In [ ]:
rec(10, "emotion025.wav", "その会社の急速な成長は、その独特な戦略によるものだった。", "そのかいしゃのきゅうそくなせーちょーわ、そのどくとくなせんりゃくによるものだった。")

In [ ]:
rec(5, "emotion026.wav", "私はいつもミネラルウォーターを持ち歩いています。", "わたしわいつもみねらるうぉーたーをもちあるいています。")

In [ ]:
rec(10, "emotion027.wav", "彼女はハンドバッグを開けて家の鍵を探してみたが、見つからなかった。", "かのじょわはんどばっぐをあけていえのかぎをさがしてみたが、みつからなかった。")

In [ ]:
rec(7, "emotion028.wav", "彼女はスタッフをまとめていけると思いますか？", "かのじょわすたっふをまとめていけるとおもいますか？")

In [ ]:
rec(10, "emotion029.wav", "牛乳はあなたの体に良いだろう、毎日飲んだほうがいい。", "ぎゅうにゅうわあなたのからだによいだろー、まいにちのんだほーがいい。")

In [ ]:
rec(10, "emotion030.wav", "あなたは流感になりかけているか、もっと重い病気かもしれません。", "あなたわりゅうかんになりかけているか、もっとおもいびょーきかもしれません。")

In [ ]:
rec(10, "emotion031.wav", "彼女はその事件を、生き生きとした筆致で描写した。", "かのじょわそのじけんを、いきいきとしたひっちでびょーしゃした。")

In [ ]:
rec(10, "emotion032.wav", "彼は自らの生涯を、インドでの病人の治療に捧げるつもりだ。", "かれわみずからのしょーがいを、いんどでのびょーにんのちりょーにささげるつもりだ。")

In [ ]:
rec(10, "emotion033.wav", "奇妙な男で、彼は人から話し掛けられないと口をきかない。", "きみょーなおとこで、かれわひとからはなしかけられないとくちをきかない。")

In [ ]:
rec(7, "emotion034.wav", "我々はその山脈で土着のガイドを雇った。", "われわれわそのさんみゃくでどちゃくのがいどをやとった。")

In [ ]:
rec(5, "emotion035.wav", "彼女は息子に家で行儀よくするように言った。", "かのじょわむすこにいえでぎょーぎよくするよーにいった。")

In [ ]:
rec(7, "emotion036.wav", "彼はライフルを拾い上げ、それで標的をねらった。", "かれわらいふるをひろいあげ、それでひょーてきをねらった。")

In [ ]:
rec(5, "emotion037.wav", "私はこの本に八百円を払った。", "わたしわこのほんにはっぴゃくえんをはらった。")

In [ ]:
rec(10, "emotion038.wav", "気分が悪くて入院したが、結果的にはたいしたことはなかった。", "きぶんがわるくてにゅういんしたが、けっかてきにわたいしたことわなかった。")

In [ ]:
rec(15, "emotion039.wav", "アフィ狙いの釣り記事ですね。英語関係のコミュのあちこちにマルチポストしています。", "あふぃねらいのつりきじですね。えーごかんけーのこみゅのあちこちにまるちぽすとしています。")

In [ ]:
rec(10, "emotion040.wav", "トラベラーズチェックを現金に替えてくれるのはこの窓口ですか？", "とらべらーずちぇっくをげんきんにかえてくれるのわこのまどぐちですか？")

In [ ]:
rec(7, "emotion041.wav", "本日の映画は、サウンド・オブ・ミュージックでございます。", "ほんじつのえーがわ、さうんどおぶみゅーじっくでございます。")

In [ ]:
rec(10, "emotion042.wav", "概してわれわれ日本人は、外国語を話すのに少し臆病すぎる。", "がいしてわれわれにほんじんわ、がいこくごをはなすのにすこしおくびょーすぎる。")

In [ ]:
rec(15, "emotion043.wav", "店の人は私のことを知るようになり、私はいつも同じウェイトレスに応対してもらっていた。", "みせのひとわわたしのことをしるよーになり、わたしわいつもおなじうぇいとれすにおーたいしてもらっていた。")

In [ ]:
rec(5, "emotion044.wav", "この丘からは何百万という星が見える。", "このおかからわなんびゃくまんとゆうほしがみえる。")

In [ ]:
rec(10, "emotion045.wav", "彼女は大学に入学したら、親から経済的に独立しようと思っていた。", "かのじょわだいがくににゅうがくしたら、おやからけーざいてきにどくりつしよーとおもっていた。")

In [ ]:
rec(7, "emotion046.wav", "サブマリンのペリスコープが、水中からにょっきり突き出ていた。", "さぶまりんのぺりすこーぷが、すいちゅうからにょっきりつきでていた。")

In [ ]:
rec(5, "emotion047.wav", "別にブルマに特別な関心があるわけじゃない。", "べつにぶるまにとくべつなかんしんがあるわけじゃない。")

In [ ]:
rec(5, "emotion048.wav", "そういうフェティシズムはないと思う。", "そーゆうふぇてぃしずむわないとおもう。")

In [ ]:
rec(15, "emotion049.wav", "ウッド夫人が作ってくれるおいしい田舎風の料理を食べたし、ミルクをたくさん飲みました。", "うっどふじんがつくってくれるおいしいいなかふうのりょーりをたべたし、みるくをたくさんのみました。")

In [ ]:
rec(15, "emotion050.wav", "私の妻が瓶をわってしまったので、台所の床は牛乳だらけになっている。", "わたしのつまがびんをわってしまったので、だいどころのゆかわぎゅうにゅうだらけになっている。")

In [ ]:
rec(10, "emotion051.wav", "喧嘩をしていた二人の子供は、お互いにしかめっ面をして座っていた。", "けんかをしていたふたりのこどもわ、おたがいにしかめっつらをしてすわっていた。")

In [ ]:
rec(15, "emotion052.wav", "事業を継続しながら、事業が依拠している不動産を、切り売りしていくことなど非現実的なのだ。", "じぎょーをけーぞくしながら、じぎょーがいきょしているふどーさんを、きりうりしていくことなどひげんじつてきなのだ。")

In [ ]:
rec(15, "emotion053.wav", "しかしペパーバーグは、そのオウムを研究することによって、動物に対する考え方が変わったと言っている。", "しかしぺぱーばーぐわ、そのおーむをけんきゅうすることによって、どーぶつにたいするかんがえかたがかわったといっている。")

In [ ]:
rec(10, "emotion054.wav", "パックマンがある条件を満たすと、追ってくるモンスターを逆襲して食べることができる。", "ぱっくまんがあるじょーけんをみたすと、おってくるもんすたーをぎゃくしゅうしてたべることができる。")

In [ ]:
rec(15, "emotion055.wav", "その数百年後に、フォークが西洋のテーブルに現れることになるが、ただちに受け入れられたわけではなかった。", "そのすうひゃくねんごに、ふぉーくがせーよーのてーぶるにあらわれることになるが、ただちにうけいれられたわけでわなかった。")

In [ ]:
rec(10, "emotion056.wav", "トップのリーダーは犬の行動学ではアルファと呼ばれ、以下ベータ、ガンマと続きます。", "とっぷのりーだーわいぬのこーどーがくでわあるふぁとよばれ、いかべーた、がんまとつづきます。")

In [ ]:
rec(7, "emotion057.wav", "ウィーンまでは歩くとどのくらいかかりますか？", "うぃーんまでわあるくとどのくらいかかりますか？")

In [ ]:
rec(5, "emotion058.wav", "すみません、この辺に詳しくないんです。", "すみません、このへんにくわしくないんです。")

In [ ]:
rec(15, "emotion059.wav", "目標は授業設計をするときの、学生の思考を触発するメディア教材の選択および、活用方法について理解することである。", "もくひょーわじゅぎょーせっけーをするときの、がくせーのしこーをしょくはつするめでぃあきょーざいのせんたくおよび、かつよーほーほーについてりかいすることである。")

In [ ]:
rec(10, "emotion060.wav", "とりあえず店の前、掃除しといてくれ。打ち水も頼む。", "とりあえずみせのまえ、そーじしといてくれ。うちみずもたのむ。")

In [ ]:
rec(10, "emotion061.wav", "人々はトーナメントが始まる何ヶ月も前に、これらの入場券を買う。", "ひとびとわとーなめんとがはじまるなんかげつもまえに、これらのにゅうじょーけんをかう。")

In [ ]:
rec(10, "emotion062.wav", "彼女の魅力は言葉では表現できない、とその芸術家は叫んだ。", "かのじょのみりょくわことばでわひょーげんできない、とそのげーじゅつかわさけんだ。")

In [ ]:
rec(15, "emotion063.wav", "事象として簡単なことを、いかにも難しそうに表現する人は、あまり頭がよさそうではない。", "じしょーとしてかんたんなことを、いかにもむずかしそーにひょーげんするひとわ、あまりあたまがよさそーでわない。")

In [ ]:
rec(15, "emotion064.wav", "デザインも、アーチ型のロゴデザインにより、現代的で登場感、躍動感あるものに仕上げました。", "でざいんも、あーちがたのろごでざいんにより、げんだいてきでとーじょーかん、やくどーかんあるものにしあげました。")

In [ ]:
rec(7, "emotion065.wav", "そんなに慌てて運転して、一体どこへ行こうってんだよ。", "そんなにあわててうんてんして、いったいどこえいこーってんだよ。")

In [ ]:
rec(7, "emotion066.wav", "時間はあるんだから、安全運転してくれよ。", "じかんわあるんだから、あんぜんうんてんしてくれよ。")

In [ ]:
rec(15, "emotion067.wav", "ディスプレイはモニタともいい、コンピュータなどの機器から出力される静止画、または動画の映像信号を表示する機器である。", "でぃすぷれいわもにたともいい、こんぴゅーたなどのききからしゅつりょくされるせーしが、またわどーがのえーぞーしんごーをひょーじするききである。")

In [ ]:
rec(15, "emotion068.wav", "撃ち合いが少し静まったとき、パパが走ってフラットに行って、私たちにサンドイッチを持ってきてくれたわ。", "うちあいがすこししずまったとき、ぱぱがはしってふらっとにいって、わたしたちにさんどいっちをもってきてくれたわ。")

In [ ]:
rec(15, "emotion069.wav", "ギリシャのフットボールの試合では、一方のチームの選手は、相手チームの陣地のラインの向こう側にボールを持ち込もうとしたのです。", "ぎりしゃのふっとぼーるのしあいでわ、いっぽーのちーむのせんしゅわ、あいてちーむのじんちのらいんのむこーがわにぼーるをもちこもーとしたのです。")

In [ ]:
rec(15, "emotion070.wav", "泳者のシンディー・ニコラスは、へとへとになって泳ぎ切った後、ドーバーでかろうじて陸に上がってきたが、海峡水泳協会のスポークスマンは、彼女がとても元気であると発表した。", "えーしゃのしんでぃーにこらすわ、へとへとになっておよぎきったあと、どーばーでかろーじてりくにあがってきたが、かいきょーすいえーきょーかいのすぽーくすまんわ、かのじょがとてもげんきであるとはっぴょーした。")

In [ ]:
rec(10, "emotion071.wav", "若い男女が人里離れた洋館で恐怖の一夜を過ごすという、ホラーの定番スタイルだ。", "わかいだんじょがひとざとはなれたよーかんできょーふのいちやをすごすとゆう、ほらーのてーばんすたいるだ。")

In [ ]:
rec(15, "emotion072.wav", "一つには、西洋人ではない人々が自分たち独自の文化に誇りを持ち始めてきたためと、また一つには、フォークを使わないそれらの地域は最も高い出生率の地域を抱えているという理由から、このことは当たっている。", "ひとつにわ、せーよーじんでわないひとびとがじぶんたちどくじのぶんかにほこりをもちはじめてきたためと、またひとつにわ、ふぉーくをつかわないそれらのちいきわもっともたかいしゅっしょーりつのちいきをかかえているとゆうりゆうから、このことわあたっている。")

In [ ]:
rec(7, "emotion073.wav", "私たちは恐怖の中で生きていて、苦しんでいるの。", "わたしたちわきょーふのなかでいきていて、くるしんでいるの。")

In [ ]:
rec(10, "emotion074.wav", "太陽も花も楽しめないし、私たちの子供時代も楽しめないのよ。", "たいよーもはなもたのしめないし、わたしたちのこどもじだいもたのしめないのよ。")

In [ ]:
rec(15, "emotion075.wav", "弊社のエンジニアが日本国内で販売されている同様の製品と仕様を比較した結果、非常に競合力があると判断いたしました。", "へーしゃのえんじにあがにほんこくないではんばいされているどーよーのせーひんとしよーをひかくしたけっか、ひじょーにきょーごーりょくがあるとはんだんいたしました。")

In [ ]:
rec(15, "emotion076.wav", "ノースウエスタン大学の研究者、アイリーン・ペパーバーグは、オウムは人の口まねをするだけでなく言葉の意味を学ぶことができることを発見しつつある。", "のーすうえすたんだいがくのけんきゅうしゃ、あいりーんぺぱーばーぐわ、おーむわひとのくちまねをするだけでなくことばのいみをまなぶことができることをはっけんしつつある。")

In [ ]:
rec(5, "emotion077.wav", "名前をテョといいます。", "なまえをてょといいます。")

In [ ]:
rec(5, "emotion078.wav", "ぴゅうぴゅう風が吹きこんでくる。", "ぴゅうぴゅうかぜがふきこんでくる。")

In [ ]:
rec(5, "emotion079.wav", "キェー。ギェー。イェーイ。", "きぇー。ぎぇー。いぇーい。")

In [ ]:
rec(5, "emotion080.wav", "ひぇーん。びぇーん。ぴぇーん。", "ひぇーん。びぇーん。ぴぇーん。")

In [ ]:
rec(5, "emotion081.wav", "プリミェー。ミェルン。ニェン。", "ぷりみぇー。みぇるん。にぇん。")

In [ ]:
rec(5, "emotion082.wav", "ステューデント。ダブルデュアル。", "すてゅーでんと。だぶるでゅある。")

In [ ]:
rec(5, "emotion083.wav", "エテュード。エデュケーショナル。サブスティテューション。", "えてゅーど。えでゅけーしょなる。さぶすてぃてゅーしょん。")

In [ ]:
rec(3, "emotion084.wav", "ジャデャクシュ。", "じゃでゃくしゅ。")

In [ ]:
rec(3, "emotion085.wav", "ブレンドデョート。", "ぶれんどでょーと。")

In [ ]:
rec(3, "emotion086.wav", "イデュスルファーゼ。", "いでゅするふぁーぜ。")

In [ ]:
rec(3, "emotion087.wav", "あっあの。", "あっあの。")

In [ ]:
rec(3, "emotion088.wav", "いっいえ。", "いっいえ。")

In [ ]:
rec(3, "emotion089.wav", "えっえぇ。", "えっえぇ。")

In [ ]:
rec(3, "emotion090.wav", "おっおい。", "おっおい。")

In [ ]:
rec(3, "emotion091.wav", "んーとね。", "んーとね。")

In [ ]:
rec(3, "emotion092.wav", "アンツィオ。", "あんつぃお。")

In [ ]:
rec(3, "emotion093.wav", "エンツォ。", "えんつぉ。")

In [ ]:
rec(3, "emotion094.wav", "カートゥーン。", "かーとぅーん。")

In [ ]:
rec(3, "emotion095.wav", "スィーディー。", "すぃーでぃー。")

In [ ]:
rec(3, "emotion096.wav", "ズィーブラ。", "ずぃーぶら。")

In [ ]:
rec(3, "emotion097.wav", "デピュティーガバナー。", "でぴゅてぃーがばなー。")

In [ ]:
rec(3, "emotion098.wav", "エリュシオン。", "えりゅしおん。")

In [ ]:
rec(3, "emotion099.wav", "ガリェント。", "がりぇんと。")

In [ ]:
rec(3, "emotion100.wav", "ラーテャン。", "らーてゃん。")